In [ ]:
%pylab inline

'''
Nearly all of the following code is based off the following paper:
http://www.flam3.com/flame_draves.pdf
Note that only a very few of the referenced transforms are implemented here,
but implementing more is trivial, so if you want to have fun, have at it!
'''

# This section is purely setup and imports. Note that not all imports

import numpy as np
import scipy as sc
import scipy.misc
import scipy.ndimage
import scipy.interpolate
import theano
#import theano.tensor as T
#from theano import shared
from tqdm import tqdm
import random
import pickle

from IPython.display import Image

In [ ]:
default_condense_args = {
    'xmin': -1,
    'xmax': 1,
    'ymin': -1,
    'ymax': 1,
    'xres': 1440,
    'yres': 1080,
    'gamma': 2.0,
    'vibrancy': 0.0,
    'supersample': 2,
    'saturation': None, # or a real value
    'contrast': None, # or a real value
    'threads_per_block': 100,
    'length': 100,
    'n': 1000000,
}
make_get = lambda d: lambda k: d[k] if k in d else default_condense_args[k]
theano_dtype = 'float32'

def make_condense(**condensed_args):
    '''
    Take a 2D histogram of where points have landed
    This is actually a wrapper function to store the parameters for the histogram, just so
    we're not re-extracting parameters every iteration (not really a big deal, but still)
    '''
    get = make_get(condensed_args)
    xmin = get('xmin')
    xmax = get('xmax')
    ymin = get('ymin')
    ymax = get('ymax')
    xres = get('xres')
    yres = get('yres')
    supersample = get('supersample')
    def condense(x, y, c):
        return np.stack([np.histogram2d(x, y, weights=c[:,channel],
                                        bins=[yres*supersample, xres*supersample],
                                        range=[[ymin, ymax], [xmin, xmax]])[0]
                         for channel in range(4)],
                        axis=2)
    return condense
    

def prepare_image(condensed, **condensed_args):
    '''
    Given a linear histogram of where points have landed, render that to an image
    Condensed is a 3D matrix of shape HEIGHT x WIDTH x 4 (RGBA)
    '''
    get = make_get(condensed_args)
    gamma = get('gamma')
    vibrancy = get('vibrancy')
    
    alph = np.expand_dims(condensed[:,:,-1], 2)
    rgb = condensed[:,:,:-1]
    alph = alph + 1
    
    # Some pixels have never had a point land in them, which causes computational issues later
    # Create and store a mask of all such pixels so they can be corrected as needed
    #mask = condensed == 0
    #mask = np.prod(mask, axis=2)
    #mask = np.logical_not(np.stack([mask] * 4, axis=2))
    #if np.any(mask):
    #    condensed[mask] = condensed[mask] - np.min(condensed[mask])
    #condensed[~mask] = 0
    #mask[:,:,:-1] |= True
    #condensed[~mask] = 1
    
    # Scale colors by log(alpha)/alpha
    color_scale = np.log(alph) / alph
    rgb = rgb * color_scale
    alph = alph * color_scale + 1
    
    # Apply gamma correction
    per_color = rgb ** (1 / gamma)
    vibrancy_factors = alph ** (1 / gamma)
    vibrancy_factors = np.log(vibrancy_factors) / vibrancy_factors
    vibrant = rgb * vibrancy_factors
    rgb = vibrant * vibrancy + per_color * (1 - vibrancy)
    
    condensed = np.concatenate([rgb, alph], axis=2)
    
    # Apply saturation boost if requested
    if get('saturation') is not None or get('contrast') is not None:
        condensed = saturate(condensed, **condensed_args)
        
    # Resolve supersampling
    scale = 1/get('supersample')
    condensed = scipy.ndimage.zoom(condensed, zoom=(scale, scale, 1))
    
    return condensed

def make_renderer(verbose=False, **condensed_args):
    import theano
    import theano.tensor as T
    
    t_condensed = T.tensor3(dtype='float32')
    condensed = _theano_renderer(t_condensed, **condensed_args)
    
    renderer = theano.function([t_condensed], condensed, allow_input_downcast=True)
    def wrapper(inp, *crap, **more_crap):
        return renderer(inp)
    if verbose:
        theano.printing.pp(renderer.maker.fgraph.outputs[0])
    return wrapper
    
def _theano_renderer(t_condensed, **condensed_args):
    import theano
    import theano.tensor as T
    from theano.tensor.signal.pool import pool_2d
    
    get = make_get(condensed_args)
    gamma = get('gamma')
    vibrancy = get('vibrancy')
    downsample = get('supersample')
    downsample = (downsample, downsample)
    
    alph = t_condensed[:,:,-1].dimshuffle(0,1,'x')
    rgb = t_condensed[:,:,:-1]
    alph += 1.0
    
    color_scale = T.log(alph) / alph
    rgb = rgb * color_scale
    alph = alph * color_scale + 1
    
    per_color = rgb ** (1 / gamma)
    vibrancy_factors = alph ** (1 / gamma)
    vibrancy_factors = T.log(vibrancy_factors) / vibrancy_factors
    vibrant = rgb * vibrancy_factors
    rgb = vibrant * vibrancy + per_color * (1 - vibrancy)
    
    condensed = T.concatenate([rgb, alph], axis=2)
    
    # Apply saturation boost if requested
    if get('saturation') is not None or get('contrast') is not None:
        condensed = theano_saturate(condensed, **condensed_args)
    
    condensed = pool_2d(condensed.T, downsample, ignore_border=True).T
    return condensed

def display_flame(condensed, renderer=prepare_image, **condensed_args):
    if condensed is None:
        raise RuntimeError("Cannot draw None")
    get = make_get(condensed_args)
    xmin = get('xmin')
    xmax = get('xmax')
    ymin = get('ymin')
    ymax = get('ymax')
    if renderer is not None:
        condensed = renderer(condensed, **condensed_args)
    fixed = condensed[:,:,:-1]
    if np.any(np.isnan(fixed)) and np.any(~np.isnan(fixed)):
        fixed[np.isnan(fixed)] = np.min(fixed[~np.isnan(fixed)])
    fixed = (fixed - fixed.min()) / fixed.ptp()
    #print(fixed)
    
    plt.figure(figsize=(10, 14))
    # Work-around, until I figure out what's up with pyplot
    plt.imshow(fixed, aspect='equal', interpolation='bicubic')
    #sc.misc.imsave('tmp.png', fixed, format='png')
    #plt.imshow(plt.imread('tmp.png'), aspect='equal', interpolation='nearest')
    plt.show(block=False)
    
def save_flame(condensed, name, transes=None, renderer=prepare_image, **condensed_args):
    if transes is not None:
        pkl = '.pkl' if not name.endswith('.pkl') else ''
        pickle.dump(transes, open(name + pkl, 'wb'))
    if renderer is not None:
        condensed = renderer(condensed, **condensed_args)
    mask = np.isnan(condensed[:,:,:3])
    condensed[:,:,:3][mask] = np.min(condensed[:,:,:3][~mask])
    png = '.png' if not name.endswith('.png') else ''
    jpg = '.jpg' if not name.endswith('.jpg') else ''
    sc.misc.imsave(name + png, condensed)
    sc.misc.imsave(name + jpg, condensed)

In [ ]:
# Skip over this, I've just put this here to get it out of the relevant code
# These functions are purely image modifications to bring out more color
# They have nothing to do with the interesting parts of rendering the fractal

def saturate(condensed, **condensed_args):
    '''
    Boost the saturation of image x by x^(1/saturation)
    '''
    saturation = make_get(condensed_args)('saturation')
    contrast = make_get(condensed_args)('contrast')
    
    # Convert from RGB to HSL
    size = tuple(condensed.shape[:2])
    rgb = condensed[:,:,:3]
    a = condensed[:,:,3]
    
    a = (a - np.min([0, np.min(a)])) / np.max([np.max(a), np.ptp(a)])
    rgb = (rgb - np.min([0, np.min(rgb)])) / np.max([np.max(rgb), np.ptp(rgb)])
    
    #http://www.niwa.nu/2013/05/math-behind-colorspace-conversions-rgb-hsl/
    r = rgb[:,:,0]
    g = rgb[:,:,1]
    b = rgb[:,:,2]
    
    mn = np.min(rgb, axis=2)
    mx = np.max(rgb, axis=2)
    
    l = (mn + mx) / 2
    
    s1 = (mx-mn)/(mx+mn)
    s2 = (mx-mn)/(2-mx-mn)
    s = np.zeros(size)
    s[l < 0.5] = s1[l < 0.5]
    s[l >= 0.5] = s2[l >= 0.5]
    
    h = np.zeros(size)
    r_max = r == mx
    g_max = g == mx
    b_max = b == mx
    h[r_max] = 0 + (g[r_max] - b[r_max]) / (mx[r_max] - mn[r_max])
    h[g_max] = 2 + (b[g_max] - r[g_max]) / (mx[g_max] - mn[g_max])
    h[b_max] = 4 + (r[b_max] - g[b_max]) / (mx[b_max] - mn[b_max])
    h /= 6
    h[np.isnan(h)] = 0
    
    # Boost channels
    if saturation is not None:
        s = s ** (1 / saturation)
    if contrast:
        half = np.ptp(l) / 2
        mean = np.min(l) + half
        l = (l - mean) / half
        l[l > 0] = l[l > 0] ** (1 / contrast)
        l[l < 0] = -((-l[l < 0]) ** (1 / contrast))
        l = l * half + mean
    
    # Convert from HSL to RGB
    #http://www.niwa.nu/2013/05/math-behind-colorspace-conversions-rgb-hsl/
    t11 = l * (1 + s)
    t12 = l + s - l * s
    t1 = np.zeros(size)
    t1[l < 0.5] = t11[l < 0.5]
    t1[l >= 0.5] = t12[l >= 0.5]
    t2 = 2 * l - t1
    
    def lim(t):
        t = np.copy(t)
        t[t > 1] -= 1
        t[t < 0] += 1
        return t
    tr = lim(h + (1/3))
    tg = lim(h)
    tb = lim(h - (1/3))
    
    def run_test(tmp):
        test1 = (6 * tmp) <= 1
        test2 = ~test1 & ((2 * tmp) <= 1)
        test3 = ~test1 & ~test2 & (3 * tmp <= 2)
        res = np.ones(size) * t2
        res[test1] = t2[test1] + (t1[test1] - t2[test1]) * 6 * tmp[test1]
        res[test2] = t1[test2]
        res[test3] = t2[test3] + (t1[test3] - t2[test3]) * (2/3 - tmp[test3]) * 6
        res[s <= 0] = l[s <= 0]
        return res
    r = run_test(tr)
    g = run_test(tg)
    b = run_test(tb)
    
    return np.stack([r, g, b, a], axis=2)

def theano_saturate(t_condensed, **condensed_args):
    '''
    Boost the saturation of image x by x^(1/saturation)
    '''
    import theano
    import theano.tensor as T
    saturation = make_get(condensed_args)('saturation')
    contrast = make_get(condensed_args)('contrast')
    
    # Convert from RGB to HSL
    rgb = t_condensed[:,:,:-1]
    a = t_condensed[:,:,-1]
    
    a = (a - T.min([0, T.min(a)])) / T.max([T.max(a), T.ptp(a)])
    rgb = (rgb - T.min([0, T.min(rgb)])) / T.max([T.max(rgb), T.ptp(rgb)])
    
    #http://www.niwa.nu/2013/05/math-behind-colorspace-conversions-rgb-hsl/
    #mn = T.min([0, T.min(rgb)])
    #ptp = T.max([T.max(rgb), T.ptp(rgb)])
    #rgb = (rgb-mn) / ptp
    r = rgb[:,:,0]
    g = rgb[:,:,1]
    b = rgb[:,:,2]
    
    mn = T.min(rgb, axis=2)
    mx = T.max(rgb, axis=2)
    
    l = (mn + mx) / 2
    
    s1 = (mx-mn)/(mx+mn)
    s2 = (mx-mn)/(2-mx-mn)
    s = T.switch(T.lt(l, 0.5), s1, s2)
    
    h = T.zeros_like(a)
    r_max = T.eq(r, mx)
    g_max = T.eq(g, mx)
    b_max = T.eq(b, mx)
    h1 = 0 + (g - b) / (mx - mn)
    h2 = 2 + (b - r) / (mx - mn)
    h3 = 4 + (r - g) / (mx - mn)
    h = T.switch(r_max, h1, T.switch(g_max, h2, T.switch(b_max, h3, h)))
    h /= 6
    h = T.switch(T.isnan(h), 0, h)
    
    # Boost channels
    if saturation is not None:
        s = s ** (1 / saturation)
    if contrast:
        half = T.ptp(l) / 2
        mean = T.min(l) + half
        l = (l - mean) / half
        l1 = l ** (1 / contrast)
        l2 = -((-l) ** (1 / contrast))
        l = T.switch(T.lt(l, 0), l2, l1)
        l = l * half + mean
    
    # Convert from HSL to RGB
    #http://www.niwa.nu/2013/05/math-behind-colorspace-conversions-rgb-hsl/
    t11 = l * (1 + s)
    t12 = l + s - l * s
    t1 = T.switch(T.lt(l, 0.5), t11, t12)
    t2 = 2 * l - t1
    
    def lim(t):
        t = t.copy()
        t = T.switch(T.gt(t, 1), t-1, t)
        t = T.switch(T.lt(t, 1), t+1, t)
        return t
    tr = lim(h + (1/3))
    tg = lim(h)
    tb = lim(h - (1/3))
    
    def run_test(tmp):
        test1 = T.le((6 * tmp), 1)
        test2 = ~test1 & T.le((2 * tmp), 1)
        test3 = ~test1 & ~test2 & T.le((3 * tmp), 2)
        res = T.ones_like(tmp) * t2
        res1 = t2 + (t1 - t2) * 6 * tmp
        res2 = t1
        res3 = t2 + (t1 - t2) * (2/3 - tmp) * 6
        res = T.switch(test1, res1, T.switch(test2, res2, T.switch(test3, res3, res)))
        res = T.switch(T.le(s, 0), l, res)
        return res
    r = run_test(tr)
    g = run_test(tg)
    b = run_test(tb)
    
    return T.stack([r, g, b, a], axis=2)


In [ ]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import theano.misc.pycuda_init
import theano.sandbox.cuda as cuda
import theano.tensor as T

class PyCUDAHist2D(theano.Op):
    __props__ = ('n', 'xres', 'yres', 'xmin', 'xptp', 'ymin', 'yptp', 'num_chans', 'length')
    
    def __init__(self, **condensed_args):
        get = make_get(condensed_args)
        self.args = {
            'n': get('n'),
            'xres': get('xres') * get('supersample'),
            'yres': get('yres') * get('supersample'),
            'xmin': float(get('xmin')),
            'xptp': float(get('xmax') - get('xmin')),
            'ymin': float(get('ymin')),
            'yptp': float(get('ymax') - get('ymin')),
            'num_chans': condensed_args.get('num_chans', 4),
            'length': condensed_args.get('length', np.min([1000, int(np.sqrt(get('n')))])),
        }
        self.n = self.args['n']
        self.xres = self.args['xres']
        self.yres = self.args['yres']
        self.xmin = self.args['xmin']
        self.xptp = self.args['xptp']
        self.ymin = self.args['ymin']
        self.yptp = self.args['yptp']
        self.num_chans = self.args['num_chans']
        self.length = self.args['length']
        
    
    def make_node(self, x, y, w):
        x = cuda.basic_ops.gpu_contiguous(cuda.basic_ops.as_cuda_ndarray_variable(x))
        y = cuda.basic_ops.gpu_contiguous(cuda.basic_ops.as_cuda_ndarray_variable(y))
        w = cuda.basic_ops.gpu_contiguous(cuda.basic_ops.as_cuda_ndarray_variable(w))
        out_t = cuda.CudaNdarrayType((False, False, False, False), dtype='float32')()
        assert x.dtype == 'float32'
        assert y.dtype == 'float32'
        assert w.dtype == 'float32'
        return theano.Apply(self, [x, y, w], [out_t])
        
    # Based on 
    #https://devblogs.nvidia.com/parallelforall/gpu-pro-tip-fast-histograms-using-shared-atomics-maxwell/
    def make_thunk(self, node, storage_map, _, _2):

        # Creates a histogram in BLOCKS_X pieces, which then need to be added together
        code = '''
        __global__ void histogram2d(const float *in_x, const float *in_y, const float *in_w, float *out) {{
            int start = (blockIdx.x * blockDim.x + threadIdx.x) * {length};

            float *block_out = &out[{xres} * {yres} * {num_chans} * blockIdx.x];
            //float *block_out = out;

            for(int i = 0; i < {length}; i++) {{
                float x = in_x[start + i];
                float y = in_y[start + i];
                int w_idx = (start + i) * {num_chans};

                int xbin = (int) (((x - {xmin}) / {xptp}) * {xres});
                int ybin = (int) (((y - {ymin}) / {yptp}) * {yres});

                if (0 <= xbin && xbin < {xres} && 0 <= ybin && ybin < {yres}) {{
                    for(int c = 0; c < {num_chans}; c++) {{
                        atomicAdd(&block_out[(ybin * {xres} + xbin) * {num_chans} + c], in_w[w_idx + c]);
                    }}
                }}
            }}
        }}
        '''.format(**self.args)
        mod = SourceModule(code)
        cuda_hist = mod.get_function('histogram2d')
        
        inputs = [storage_map[v] for v in node.inputs]
        outputs = [storage_map[v] for v in node.outputs]
        
        _x, _y, _w = inputs
        (_out,) = outputs

        def run_hist():
            x = _x[0]
            y = _y[0]
            w = _w[0]
            n = x.size
            xres = self.args['xres']
            yres = self.args['yres']
            num_chans = self.args['num_chans']
            length = self.args['length']
            num_blocks = 2
            threads_per_block = int(n / length / num_blocks)
            if _out[0] is None or _out[0].shape != (num_blocks, yres, xres, num_chans):
                _out[0] = cuda.CudaNdarray.zeros((num_blocks, yres, xres, num_chans))
            cuda_hist(x, y, w, _out[0], block=(threads_per_block,1,1), grid=(num_blocks,1))
            
        return run_hist#, zeros((num_blocks, yres, xres, num_chans), 'float32')
    
    

In [ ]:
theanoify = True
use_gpu = True

def chaos_game_cpu(fs, f_final=None, n=1000000, n_runs=1, iters=200, min_iter=20, and_display=True, resume=None, seed=None, **condense_args):
    '''
    The Chaos Game is simple (https://en.wikipedia.org/wiki/Chaos_game)
    1) Pick a random point (x,y)
    2) Apply a random transform F to it
    3) Record where it lands (in terms of pixels)
    4) Repeat a lot
    The Flame algorithm tweaks this to include 3 color channels per point
    This results in four histograms, one for each color channel, one for alpha (alpha for a point is 1)
    This implementation is a vectorized approach that runs n points simultaneously
    Note that the result is the same, since for a single iteration, the function applied to a point
    is still randomly chosen per-point
    '''
    condense_args['n'] = n
    
    condensed = resume
    fs = [(var, weight) for f in fs for var, weight in f]
    fs, weights = zip(*fs)
    if theanoify:
        import theano
        import theano.tensor as T
        t_x = T.vector(dtype='float32')
        t_y = T.vector(dtype='float32')
        fs = [f(t_x, t_y) for f in fs]
        fs = [theano.function([t_x, t_y], [out_x, out_y], allow_input_downcast=True) for out_x, out_y in fs]
    # Launching with the same parameters should result in the same color scheming
    np.random.seed((hash(tuple(fs)) % 0xffffffff) if seed is None else seed)
    weights /= np.sum(weights)
    num_fs = len(fs)
    range_fs = list(range(num_fs))
    # Assign a random color to each transform
    colors = np.random.uniform(low=0, high=1, size=(num_fs, 3))
    colors = np.hstack([colors, np.ones((num_fs, 1))])
    color_final = np.random.uniform(low=0, high=1, size=3)
    color_final = np.hstack([color_final, [1]])
    
    np.random.seed()
    condense = make_condense(**condense_args)
    from time import time
    for run in range(n_runs):
        # Pick n random points
        x = np.random.uniform(low=-1.0, high=1.0, size=n)
        y = np.random.uniform(low=-1.0, high=1.0, size=n)
        xf = np.copy(x)
        yf = np.copy(y)
        # Assign each a random starting color
        c = np.random.uniform(low=0,    high=1.0, size=(n, 4))
        cf = np.copy(c)
        for it in tqdm(range(iters)):
            s1 = time()
            # Pick a transform for each point
            f_choices = np.random.choice(range_fs, p=weights, replace=True, size=n)
            for f in range(num_fs):
                # For all points affected by transform f, apply f
                sel = f_choices == f
                x[sel], y[sel] = fs[f](x[sel], y[sel])
                if f_final is not None:
                    xf[sel], yf[sel] = f_final(x[sel], y[sel])
                else:
                    xf[sel], yf[sel] = x[sel], y[sel]
                # Shift these points' colors toward f's color
                c[sel] = (c[sel] + colors[f]) / 2
                # Shift skew toward a universal color
                cf[sel] = (c[sel] + color_final) / 2
            s2 = time()
            # Render this iteration of points
            if it >= min_iter:
                condensed = condense(xf, yf, cf) + (condensed if condensed is not None else 0)
            s3 = time()
            print("Times: %f, %f" % (s2-s1, s3-s2))
            if type(and_display) == int and it >= min_iter and it % and_display == 0:
                if use_gpu:
                    condensed = buffer.sum(0).get()
                display_flame(condensed, **condense_args)
    if and_display:
        display_flame(condensed, **condense_args)
    return condensed

def chaos_game(fs, f_final=None, n=1000000, n_runs=1, iters=200, min_iter=20, and_display=True, resume=None, reuse_func=None, seed=None, **condense_args):
    '''
    The Chaos Game is simple (https://en.wikipedia.org/wiki/Chaos_game)
    1) Pick a random point (x,y)
    2) Apply a random transform F to it
    3) Record where it lands (in terms of pixels)
    4) Repeat a lot
    The Flame algorithm tweaks this to include 3 color channels per point
    This results in four histograms, one for each color channel, one for alpha (alpha for a point is 1)
    This implementation is a vectorized approach that runs n points simultaneously
    Note that the result is the same, since for a single iteration, the function applied to a point
    is still randomly chosen per-point
    '''
    import theano
    import theano.tensor as T
    #color_random = lambda size: np.random.uniform(low=0, high=1, size=size)
    color_random = lambda size: np.random.beta(0.3, 0.3, size=size)

    cast = np.cast['float32']
    
    fs = [(var, weight) for f in fs for var, weight in f]
    num_fs = len(fs)
    fs, weights = zip(*fs)
    weights /= np.sum(weights)
        
    if reuse_func is None:
        condense_args['n'] = n
        if resume is not None:
            condensed = theano.shared(resume)
        else:
            condensed = theano.shared(cuda.CudaNdarray(cast(np.zeros((1,1,1,1)))), name='condensed')

        np.random.seed((hash(tuple(fs)) % 0xffffffff) if seed is None else seed)
        # Assign a random color to each transform
        colors = color_random((num_fs, 3))
        colors = np.hstack([colors, np.ones((num_fs, 1))])
        colors = cuda.CudaNdarray(cast(colors), dtype='float32')

        color_final = color_random(3)
        color_final = np.hstack([color_final, [1]])
        color_final = cuda.CudaNdarray(cast(color_final), dtype='float32')

        np.random.seed(seed)
        # Pick n random points
        x = np.random.uniform(low=-1.0, high=1.0, size=n)
        y = np.random.uniform(low=-1.0, high=1.0, size=n)
        # Assign each a random starting color
        #c = np.random.uniform(low=0,    high=1.0, size=(n, 4))
        c = color_random((n,4))

        t_x = theano.shared(cuda.CudaNdarray(cast(x), dtype='float32'), name='x')
        t_y = theano.shared(cuda.CudaNdarray(cast(y), dtype='float32'), name='y')
        t_c = theano.shared(cuda.CudaNdarray(cast(c), dtype='float32'), name='c')

        for f in fs:
            __x, __y = f(t_x, t_y)
            print(str(f))
            print("{}, {}".format(__x.type(), __y.type()))

        condensor_op = PyCUDAHist2D(**condense_args)

        t_choices = T.ivector('choices')

        #new_x, new_y, new_c = t_x, t_y, t_c
        new_xs, new_ys, new_cs = [], [], []
        for f in range(num_fs):
            sel = T.eq(t_choices, f).nonzero()
            tmp_x, tmp_y = fs[f](t_x[sel], t_y[sel])
            new_xs.append(tmp_x)
            new_ys.append(tmp_y)
            new_cs.append((t_c[sel] + colors[f]) / 2)

        new_x = T.concatenate(new_xs, axis=0)
        new_y = T.concatenate(new_ys, axis=0)
        new_c = T.concatenate(new_cs, axis=0)

        cf = (new_c + color_final) / 2
        t_condense = condensor_op(new_x, new_y, cf)
        t_condense = theano.ifelse.ifelse(condensed.size > 1, t_condense + condensed, t_condense)
        t_condense_final = t_condense.sum(axis=0, keepdims=False)
        rendered = _theano_renderer(t_condense_final, **condense_args)

        pre_step_function = theano.function([t_choices], [], updates=[(t_x, new_x), (t_y, new_y), (t_c, new_c)])
        step_function = theano.function([t_choices], [], updates=[(condensed, t_condense), (t_x, new_x), (t_y, new_y), (t_c, new_c)])
        _create_image = theano.function([t_condense], [rendered])
        reuse_func = (pre_step_function, step_function, _create_image, condensed, t_x, t_y, t_c)
    else:
        pre_step_function, step_function, _create_image, condensed, t_x, t_y, t_c = reuse_func
        condensed.set_value(cuda.CudaNdarray(cast(np.zeros((1,1,1,1)))))
        t_x.set_value(cuda.CudaNdarray(cast(np.random.uniform(low=-1.0, high=1.0, size=n)), dtype='float32'))
        t_y.set_value(cuda.CudaNdarray(cast(np.random.uniform(low=-1.0, high=1.0, size=n)), dtype='float32'))
        t_c.set_value(cuda.CudaNdarray(cast(color_random((n,4))), dtype='float32'))
        
    create_image = lambda: np.asarray(_create_image(condensed.get_value()))[0,:,:,:]
    
    #theano.printing.debugprint(step_function)
    
    range_fs = range(num_fs)
    for run in range(n_runs):
        for it in tqdm(range(iters)):
            f_choices = np.random.choice(range_fs, p=weights, replace=True, size=n)
            if it < min_iter:
                pre_step_function(f_choices)
            else:
                step_function(f_choices)
            if type(and_display) == int and it >= min_iter and it % and_display == 0:
                res = create_image()
                display_flame(res, renderer=None, **condense_args)
    if (type(and_display) == bool and and_display) or (type(and_display) == int and it >= min_iter and it % and_display == 0):
        display_flame(create_image(), renderer=None, **condense_args)
    return condensed.get_value(), create_image(), reuse_func

#test_settings = {
#    'n': 1000000,
#    'n_runs': 1,
#    'iters': 50,
#    'xmin': -1.166,
#    'xmax': 1.166,
#    'ymin': -1,
#    'ymax': 1,
#    'and_display': True,
#    'xres': 1440,
#    'yres': 1080,
#    'supersample': 2,
#    'vibrancy': 1,
#    'saturation': 1,
#}
#transes = [rotate(np.pi)]
#cond, img = chaos_game(transes, **test_settings)

In [ ]:
class Var():
    '''
    Every variation takes the form of the following function:
    F(x, y) = V_p(a*x + b*y + c, d*x + e*y + f)
    Where V_p is some parametrized function that takes 2D points and returns 2D points
    Note that V_p does not have to be linear (and most of the ones below aren't)
    This class simplifies the definition of new variations by implementing the a-f transformation
    step, then calling V_p, while also storing what a-f and the parameters p are, thus allowing
    this class to be called simply as "my_custom_variation(x, y)", and all other parameters are
    automatically made available to V_p
    Also, since Python allows "func(*params)", you can name your extra parameters whatever you want
    E.g., def my_custom_var(x, y, a, b, c, d, e, f, theta, epsilon, foobar): ...
    Use the @variation decorator below to create Var instances from V_p functions
    NOTE: When implementing V_p, DO NOT perform the a*x+b*y+...+f transform, this is pre-applied
    to x, y before they get passed to V_p
    '''
    def __init__(self, func, num_params, a, b, c, d, e, f, *p, **kwargs):
        dargs = np.random.randn(6)
        self.args = [a, b, c, d, e, f]
        self.args = [v if v is not None else dv for v, dv in zip(self.args, dargs)]
        p = [p[i] if i < len(p) else np.random.randn() for i in range(num_params)]
        self.p = list(p)
        self.all_params = self.args + self.p
        if theanoify:
            import theano.tensor as T
            self.shared_vars = [theano.shared(np.cast['float32'](p), l, allow_downcast=True) for l, p in zip('abcdef', self.all_params)]
            s = self.shared_vars
            def shared_pre(x, y):
                return s[0] * x + s[1] * y + s[2], s[3] * x + s[4] * y + s[5]
            self.shared_pre = shared_pre
        self.func = func #jit(func)
        self.make_pre()
        self.weight = 1
        self.unchanging = kwargs.get('unchanging', False)
        #print("Initialized %s with %d parameters (%s)" % (func.__name__, num_params, ",".join("%f" % param for param in self.all_params)))
        
    def make_pre(self):
        if theanoify:
            self.all_params = np.cast['float32'](self.all_params)
            for shared_var, p_value in zip(self.shared_vars, self.all_params):
                shared_var.set_value(p_value)
        else:
            self.all_params = np.array(self.all_params)
            a, b, c, d, e, f = self.all_params[:6]
            def pre(x, y):
                return a * x + b * y + c, d * x + e * y + f
            self.pre = pre
        
    def __call__(self, x, y):
        if theanoify:
            x, y = self.shared_pre(x, y)
            return self.func(x, y, *self.shared_vars)
        else:
            x, y = self.pre(x, y)
            return self.func(x, y, *self.all_params)
    
    def __str__(self):
        return "%s (%s)" % (self.func.__name__, 
                            ",".join(str(param) for param in self.all_params))
    
    def __len__(self):
        return 1
    
    def weights(self):
        return [float(self.weight)]
    
    def __iter__(self):
        return iter([(self, float(self.weight))])
    
    def __hash__(self):
        return hash(tuple([self.func.__name__] + list(self.all_params)))
    
    def visualize(self, default_params=True):
        num = 30
        xs = np.linspace(-1, 1, num=num, endpoint=True)
        ys = xs
        xs, ys = np.meshgrid(xs, ys)
        if not default_params:
            xs, ys = self.pre(xs, ys)
        all_args = ([1,0,0,0,1,0] + [1 for p in self.p]) if default_params else self.all_args
        xs, ys = self.func(xs, ys, *all_args)
        ys *= -1 # To match how images are displayed
        fig = plt.figure(figsize=(4,4))
        style = 'b-'
        for col in range(num):
            plt.plot(xs[:,col], ys[:,col], style)
        style = 'r-'
        for row in range(num):
            plt.plot(xs[row,:], ys[row,:], style)
        plt.title(self.func.__name__)
        plt.xlim(-1.5, 1.5)
        plt.ylim(-1.5, 1.5)
        plt.show(block=False)
        
    def sample(self, **kwargs):
        '''
        This doesn't really work well... need to find a way to demonstrate a variation
        without all the points exploding away from [-1,1], and with something reliably good
        '''
        default_args = {'xres': 640, 'yres': 480, 'supersample': 1, 'and_display': True, 'n': 10000, 'iters': 100}
        default_args.update(**kwargs)
        print(self.func.__name__)
        # Repeating itself allows for multiple colors to arise
        chaos_game([self]*3, **default_args)
        
    def __getstate__(self):
        return {'p': self.p, 'a': self.args, 'f': self.func.__name__}
    
    def __setstate__(self, state):
        self.p = state['p']
        self.args = state['a']
        self.all_params = self.args + self.p
        a, b, c, d, e, f = self.args
        self.func = _all_variations[state['f']]
        def pre(x, y):
            return a * x + b * y + c, d * x + e * y + f
        self.pre = pre
        
        
_all_variations = {}
all_variations = []
def variation(func_or_num_params=None):
    '''
    This is a function decorator that can optionally be parametrized
    @variation == @variation(0), so I'll just describe @variation(n)
    Use variation to decorate a function with the following signature:
    def my_custom_var(x, y, a, b, c, d, e, f, ...)
    variation converts this into the equivalent of the following signature:
    def initialize_my_custom_var(a, b, c, d, e, f, ...):
        def my_custom_var(x, y):
            ...
        return my_custom_var
    This allows you to initialize parametric variations, and then use them as simple 2D transforms
    '''
    if isinstance(func_or_num_params, int): 
        num_params = func_or_num_params
        func = None
    else:
        num_params = 0
        func = func_or_num_params
        
    if func is None:
        def make_variation(func):
            def inner(a=None, b=None, c=None, d=None, e=None, f=None, *p, **kwargs):
                return Var(func, num_params, a, b, c, d, e, f, *p, **kwargs)
            _all_variations[func.__name__] = func
            all_variations.append(inner)
            return inner
        return make_variation
    else:
        def inner(a=None, b=None, c=None, d=None, e=None, f=None, *p, **kwargs):
            return Var(func, num_params, a, b, c, d, e, f, *p, **kwargs)
        _all_variations[func.__name__] = func
        all_variations.append(inner)
        return inner

class Transform():
    '''
    Transforms are probabilistic collections of variations, with some extra fluff
    When a transform is called with x, y, it selects a random variation and runs it for that instance
    This can also integrate symmetry, including handling selection weighting to ensure that
    symmetry is properly represented in the final result
    Note that when a transform is given to the Chaos Game, it's actually unrolled, so
    each variation inside of it is handled separately (selection probabilities are retained)
    '''
    def __init__(self, *variations, weights=None, rotational_sym=1, dihedral_sym=False):
        n = len(variations)
        if n == 0:
            raise RuntimeError("Transform must have at least one variation")
        
        if weights is None:
            weights = np.random.uniform(size=len(variations))
        elif weights == 1:
            weights = np.ones(len(variations))
        else:
            if not len(weights) == len(variations):
                raise RuntimeError("Number of variations and weights for a Transform must be equal")
        
        weights = list(weights)
        variations = list(variations)
        
        total_weight = np.sum(weights)
        if dihedral_sym:
            variations.append(dihedral())
            weights.append(total_weight)
            total_weight *= 2
            
        for rot in range(rotational_sym-1):
            theta = 2 * np.pi * (rot + 1) / rotational_sym
            variations.append(rotate(theta))
            weights.append(total_weight)
        
        self.variations = variations
        self.weights = weights / np.sum(weights)
        
        print("Initialized transform: %s" % ", ".join(str(var) for var in self.variations))
        
#     def __call__(self, x, y):
#         num_fs = len(self.variations)
#         f_choices = np.random.choice(range(num_fs), p=self.weights, size=x.shape[0])
#         for f in range(num_fs):
#             sel = f_choices == f
#             x[sel], y[sel] = self.variations[f](x[sel], y[sel])
#         return x, y
    
    def __str__(self):
        return "Transform<%s>" % (";".join(str(var) for var in self.variations))
    
    def __len__(self):
        return len(self.variations)
    
    def weights(self):
        return self.weights
    
    def __iter__(self):
        return iter([(trans, w1 * w2) for var, w1 in zip(self.variations, self.weights) for trans, w2 in var])
    
def random_transform():
    num_trans = np.random.choice([1,2,3])
    transes = []
    for t in range(num_trans):
        num_vars = np.random.randint(3,8)
        variations = np.random.choice(list(all_variations), size=num_vars, replace=True)
        variations = [var() for var in variations]
        sym = np.random.choice([0, 1, 2, 2])
        if sym == 0:
            dih = False
            rot = 1
        elif sym == 1:
            dih = True
            rot = 1
        elif sym == 2:
            dih = True
            rot = np.random.choice([2,3,4,5])
        transes.append(Transform(*variations, rotational_sym=rot, dihedral_sym=dih))
    return transes

# ==================================================
# The actual transforms
# ==================================================
if theanoify:
    import theano
    import theano.tensor as T
    from theano.tensor.shared_randomstreams import RandomStreams
    _rand_stream = RandomStreams()
    sin = T.sin
    cos = T.cos
    pi = np.cast['float32'](np.pi)
    arctan = T.arctan2
    sqrt = T.sqrt
    choice = _rand_stream.choice
    uniform = _rand_stream.uniform
    def _trunc(v):
        raise NotImplementedError()
    def _zeros(x):
        return T.zeros_like(x, dtype='float32')
else:
    sin = np.sin
    cos = np.cos
    pi = np.pi
    arctan = np.arctan2
    sqrt = np.sqrt
    choice = np.random.choice
    uniform = np.random.uniform
    def _trunc(v):
        return np.trunc(v)
    def _zeros(x):
        return np.zeros(_size(x))
def _r2(x, y):
    return x * x + y * y
def _r(x, y):
    return sqrt(_r2(x, y))
def _theta(x, y):
    return arctan(x, y)
def _phi(x, y):
    return arctan(y, x)
def _size(x):
    return x.shape[0]
def _omega(x):
    return choice([0, pi], size=_size(x))
def _alpha(x):
    return choice([-1, 1], size=_size(x))
def _unif(x):
    return uniform(low=0, high=1, size=_size(x))
    
@variation # can also be @variation() or @variation(0)
def identity(x, y, a, b, c, d, e, f):
    return x, y

rotate = lambda theta: identity(np.cos(theta), -np.sin(theta), 0, np.sin(theta), np.cos(theta), 0, unchanging=True)
dihedral = lambda: identity(-1, 0, 0, 0, 1, 0, unchanging=True)

@variation(1)
def _free_rotation(x, y, a, b, c, d, e, f, theta):
    return cos(theta) * x - sin(theta) * y, sin(theta) * x + cos(theta) * y

@variation
def sinusoidal(x, y, a, b, c, d, e, f):
    return sin(x), cos(y)

@variation
def spherical(x, y, a, b, c, d, e, f):
    ir = 1 / _r2(x, y)
    return ir * x, ir * y

@variation
def pdj(x, y, a, b, c, d, e, f):
    return sin(a * y) - cos(b * x), sin(c * x) - cos(d * y)

@variation
def handkerchief(x, y, a, b, c, d, e, f):
    r = _r(x, y)
    t = _theta(x, y)
    return r * sin(t + r), r * cos(t - r)

@variation
def disc(x, y, a, b, c, d, e, f):
    td = _theta(x, y) / pi
    r = _r(x, y)
    return td * sin(pi * r), td * cos(pi * r)

@variation
def polar(x, y, a, b, c, d, e, f):
    r = _r(x, y)
    t = _theta(x, y)
    return t / pi, r - 1

@variation
def swirl(x, y, a, b, c, d, e, f):
    r2 = _r2(x, y)
    return x * sin(r2) - y * cos(r2), x * cos(r2) + y * sin(r2)

@variation
def horseshoe(x, y, a, b, c, d, e, f):
    ir = 1 / _r(x, y)
    return ir * (x - y) * (x + y), ir * 2 * x * y

@variation
def heart(x, y, a, b, c, d, e, f):
    r = _r(x, y)
    return r * sin(pi * r), -r * cos(pi * r)

@variation
def spiral(x, y, a, b, c, d, e, f):
    r = _r(x, y)
    ir = 1 / r
    t = _theta(x, y)
    return ir * cos(t) + sin(t), ir * sin(t) - cos(r)

@variation
def hyperbolic(x, y, a, b, c, d, e, f):
    r = _r(x, y)
    t = _theta(x, y)
    return sin(t) / r, r * cos(t)

# This works fine, I just don't like it
#@variation
#def diamond(x, y, a, b, c, d, e, f):
#    r = _r(x, y)
#    t = _theta(x, y)
#    return sin(t) * cos(r), cos(t) * sin(r)

@variation
def ex(x, y, a, b, c, d, e, f):
    r = _r(x, y)
    t = _theta(x, y)
    p0 = sin(t + r)
    p1 = cos(t - r)
    return r * (p0 ** 3 + p1 ** 3), r * (p0 ** 3 - p1 ** 3)

if not theanoify:
    @variation
    def julia(x, y, a, b, c, d, e, f):
        sr = sqrt(_r(x, y))
        t = _theta(x, y)
        o = _omega(x)
        return sr * cos(t / 2 + o), sr * sin(t / 2 + o)

    @variation
    def bent(x, y, a, b, c, d, e, f):
        #c1 = (x >= 0) & (y >= 0)
        if theanoify:
            c2 = T.lt(x, 0) & T.ge(y, 0)
            c3 = T.ge(x, 0) & T.lt(y, 0)
            c4 = T.lt(x, 0) & T.lt(y, 0)
            x_shifted = x * 2
            y_shifted = y / 2
            fx = T.switch(c2 | c4, x_shifted, x)
            fy = T.switch(c3 | c4, y_shifted, y)
        else:
            c2 = (x <  0) & (y >= 0)
            c3 = (x >= 0) & (y <  0)
            c4 = (x <  0) & (y <  0)
            fx = x
            fy = y
            fx[c2 | c4] = fx[c2 | c4] * 2
            fy[c3 | c4] = fy[c3 | c4] / 2
        return fx, fy
    
@variation
def waves(x, y, a, b, c, d, e, f):
    return x + b * sin(y / (c*c)), y + e * sin(x / (f*f))

@variation
def fisheye1(x, y, a, b, c, d, e, f):
    f = 2 / (_r(x, y) + 1)
    return f * y, f * x
    

In [ ]:
# Run this to see how the variations behave
for vari in all_variations:
    vari().visualize()

In [ ]:
# Sierpinski's Gasket (https://en.wikipedia.org/wiki/Sierpinski_triangle)
# This is a common fractal that can be rendered, among other ways, using the chaos game
# I put it here to demonstrate how to do something obvious and known, and also to test the renderer
# NOTE: I don't think the Mandelbrot set can be rendered this way :'(

@variation
def gasket0(x, y, *crap):
    return x/2, y/2

@variation
def gasket1(x, y, *crap):
    return (x+1)/2, y/2

@variation
def gasket2(x, y, *crap):
    return x/2, (y+1)/2
gasket0 = gasket0(1, 0, 0, 0, 1, 0)
gasket1 = gasket1(1, 0, 0, 0, 1, 0)
gasket2 = gasket2(1, 0, 0, 0, 1, 0)

def sierpinski_gasket(**display_args):
    t = [gasket0, gasket1, gasket2]
    c, img = chaos_game(t, **display_args)
    display_flame(img, renderer=None)
    save_flame(img, 'sierpinski_gasket', renderer=None, transes=t)

sierpinski_gasket(iters=30, n=100000, xmin=0, xmax=1, ymin=0, ymax=1, xres=512, yres=512)

In [ ]:
# FINALLY, we can render some fractal flame art
# This cell sets up a new flame configuration and renders it

flame_settings = {
    'n': 1000000,
    'n_runs': 1,
    'iters': 500,
    'xmin': -1.166,
    'xmax': 1.166,
    'ymin': -1,
    'ymax': 1,
    'and_display': 100,
    'xres': 1440,
    'yres': 1080,
    'supersample': 2,
    'vibrancy': 2,
    'saturation': None,
}
transes = random_transform()
cond, img = chaos_game(transes, **flame_settings)
display_flame(img, renderer=None, **flame_settings)

In [ ]:
print(cond.max())

In [ ]:
# This cell demonstrates how to pick up where you left off and continue rendering 
# (creates new batch of points, but that doesn't really matter)
# (Make sure not to change any render parameters, only display parameters or run configurations)
flame_settings['resume'] = cond
flame_settings['n'] = 1000000
flame_settings['iters'] = 200

cond = chaos_game(transes, **flame_settings)

In [ ]:
# This cell is for playing with display parameters without changing the render
from time import time
s1 = time()
display_flame(cond, renderer=theano_renderer, **flame_settings)
print(time() - s1)
s1 = time()
display_flame(cond, **flame_settings)
print(time() - s1)

In [ ]:
import theano
theano.config.device

In [ ]:
save_flame(cond, 'flame101', transes=transes, **flame_settings)

In [ ]:
# Save the flame to an image file (saves both to png and jpg)
flame_settings = {
    'n': 1000000,
    'n_runs': 1,
    'iters': 500,
    'xmin': -1.166,
    'xmax': 1.166,
    'ymin': -1,
    'ymax': 1,
    'and_display': 100,
    'xres': 1440*2,
    'yres': 1080*2,
    'supersample': 2,
    'vibrancy': 1,
    'saturation': 3,
    'renderer': theano_renderer,
}

for i in range(52, 100):
    transes = random_transform()
    cond = chaos_game(transes, **flame_settings)
    save_flame(cond, 'flame%d' % i, transes=transes, vibrancy=1.0, saturation=3.0)

In [ ]:
'''
TODO:
Performance optimization (& parallelization)
Implement density estimation
Animate
'''

In [ ]:
#def _norm(x, y):
#    return np.sqrt(x*x+y*y)
#
#def _rotate(x, y, rot):
#    n = _norm(x, y)
#    r = np.arctan2(y, x)
#    r += rot
#    return n * np.cos(r), n * np.sin(r)

class RotaryFloat():
    def __init__(self, init_value=None):
        if init_value is None:
            init_value = np.random.randn()
        self.center = np.random.randn()
        self.val = init_value
        self.dv = 0.001 * np.random.randn()
        self.deltas = [[0, 1], [0, 1], [0, 1]]
        
    def _ddv(self):
        return (-0.001 * self.deltas[2][1] + self.deltas[2][0]) * np.sign(self.val - self.center)
    
    def __float__(self):
        return self.val * self.deltas[0][1] + self.deltas[0][0]
    
    def step(self):
        self.dv += self._ddv()
        cur_dv = self.dv * self.deltas[1][1] + self.deltas[1][0]
        self.val += cur_dv
        
class PositiveRotaryFloat(RotaryFloat):
    def __float__(self):
        return np.abs(super().__float__())

class AnimatedVariation():
    def __init__(self, variation, lifespan=None):
        self.var = variation
        self.anim_args = [RotaryFloat() for p in variation.all_params]
        self.var.weight = PositiveRotaryFloat()
        #self.age = 0
        #self.lifespan = lifespan
        
    def step(self):
        if not self.var.unchanging:
            for rf in self.anim_args:
                rf.step()
            self.var.all_params = [float(rf) for rf in self.anim_args]
            self.var.make_pre()
        
        #self.age += 1
        #if self.lifespan is not None:
        #    phase = 0 if self.age >= self.lifespan else (self.age / self.lifespan)
        #    phase *= 2 * np.pi
        #    mag = (np.cos(phase) + 1) * -0.5
        #    self.var.weight = mag
        
    def __getattr__(self, key):
        return self.var.__getattribute__(key)
    
    def __str__(self):
        return "animated_" + str(self.var)
        
    def __call__(self, x, y):
        self.var(x, y)
    
    def __len__(self):
        return len(self.var)
    
    def __iter__(self):
        return iter(self.var)
    
    def __hash__(self):
        return hash(self.var)
    
class AnimatedTransform():
    def __init__(self, trans):
        pass



In [ ]:
flame_settings = {
    'n': 1000000,
    'n_runs': 1,
    'iters': 50,
    'xmin': -1.166,
    'xmax': 1.166,
    'ymin': -1,
    'ymax': 1,
    'and_display': False,
    'xres': 1440,
    'yres': 1080,
    'supersample': 1,
    'vibrancy': 1,
    'saturation': None,
}
theano.config.optimizer = 'fast_run'

f_resume = None
odd_transform = random_transform()
for t in odd_transform:
    t.variations = [AnimatedVariation(var) for var in t.variations]
    for var in t.variations:
        var.step()
for i in range(600):
    cond, img, f_resume = chaos_game(odd_transform, reuse_func=f_resume, **flame_settings)
    display_flame(img, renderer=None, **flame_settings)
    save_flame(img, 'animated_flame_%d' % i, renderer=None, **flame_settings)
    for trans in odd_transform:
        for var in trans.variations:
            var.step()
    del cond

In [ ]:
'''TODO:
More powerful animation
    - Create animated transform that fades variations in and out
    - Change animated variations to be arbitrary functions (to transform themselves)
Add music visualization
    - Compute fft over audio windows
    - Introduce arbitrary values from the fft into the variations at that time window
'''

In [ ]:
import numpy as np
from scipy.io.wavfile import read

class Music():
    def __init__(self, input_file='input.wav', framerate=15):
        sample_rate, wave = read(input_file)
        samples_per_frame = int(sample_rate / framerate)
        window_size = 2 ** (np.floor(np.log2(sample_rate)) - 1)

        pad_width = sample_rate # 1 second of 0 padding
        wave = [0] * pad_width + list(wave) + [0] * pad_width
        wave = np.array(wave)
        centers = np.arange(pad_width, len(wave) - pad_width, samples_per_frame)

        from tqdm import tqdm
        ffts = [np.real(np.fft.rfft(wave[center-window_size:center+window_size])) for center in tqdm(centers)]
        ffts = np.stack(ffts)
        ffts = np.log(ffts)
        ffts[np.isnan(ffts)] = 0
        ffts[np.isinf(ffts)] = 0
        ffts[np.isneginf(ffts)] = 0
        
        bins = np.cast['int'](np.logspace(0, np.log(ffts.shape[1]), 16+1, endpoint=True, base=np.e))
        edges = [(int(bins[i]), int(bins[i+1])) for i in range(len(bins) - 1)]
        # sqrt of the mean exp, to give high values more impact, but not outrageously much
        binned = [[np.sqrt(np.mean(np.exp(row[a:b]))/10) for a, b in edges] for row in tqdm(ffts)]
        binned = np.array(binned)
        
        self.bass = binned[:,5:8].mean(axis=1)
        self.mid = binned[:,8:10].mean(axis=1)
        self.high = binned[:,10:12].mean(axis=1)
        self.freqs = np.stack([self.bass, self.mid, self.high])
        print(self.freqs.shape)
        
        self.smoothed_bass = np.array([self.bass[i-5:i+5].mean() for i in range(len(self.bass))])
        self.smoothed_mid = np.array([self.mid[i-5:i+5].mean() for i in range(len(self.mid))])
        self.smoothed_high = np.array([self.high[i-5:i+5].mean() for i in range(len(self.high))])
        self.smoothed = np.stack([self.smoothed_bass, self.smoothed_mid, self.smoothed_high])
        print(self.smoothed.shape)
        
        self.bass_dev = binned[:,4:9].std(axis=1)
        self.mid_dev = binned[:,7:11].std(axis=1)
        self.high_dev = binned[:,9:13].std(axis=1)
        self.devs = np.stack([self.bass_dev, self.mid_dev, self.high_dev])
        print(self.devs.shape)
        
    def __len__(self):
        return len(self.bass)
    
    def visualize(self):
        from tqdm import tqdm
        x = np.arange(3)
        ylims1 = np.max(self.freqs)
        ylims2 = np.max(self.devs)
        for t in tqdm(range(len(self))):
            fig, ax1 = plt.subplots()
            ax1.plot(x, self.freqs[:, t], 'r')
            ax1.plot(x, self.smoothed[:, t], 'b')
            ax1.set_ylim(0, ylims1)
            ax1.set_ylabel('frequencies')
            ax2 = ax1.twinx()
            ax2.plot(x, self.devs[:, t], 'g')
            ax2.set_ylim(0, ylims2)
            ax2.set_ylabel('std deviations')
            plt.savefig('fft_useful_%d.png' % t)
            plt.close()

class MusicalVariation(AnimatedVariation):
    def __init__(self, input_music, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.music = input_music
        self.scale = lambda x: x/10
        self.t = 0
        
    def step(self):
        self.t += 1
        
        for rf in list(self.anim_args[0:2]) + list(self.anim_args[3:5]):
            rf.deltas[0][0] = self.scale(self.music.smoothed_bass[self.t])
            rf.deltas[2][1] = self.scale(self.music.high[self.t])
        
        super().step()
        


In [ ]:
flame_settings = {
    'n': 1000000,
    'n_runs': 1,
    'iters': 50,
    'xmin': -1.166,
    'xmax': 1.166,
    'ymin': -1,
    'ymax': 1,
    'and_display': False,
    'xres': 1440,
    'yres': 1080,
    'supersample': 2,
    'vibrancy': 1,
    'saturation': None,
}
theano.config.optimizer = 'fast_run'

f_resume = None
odd_transform = random_transform()
music = Music()
music.visualize()

for t in odd_transform:
    t.variations = [MusicalVariation(music, var) for var in t.variations]
    for var in t.variations:
        var.step()
for i in range(len(music)):
    cond, img, f_resume = chaos_game(odd_transform, reuse_func=f_resume, **flame_settings)
    display_flame(img, renderer=None, **flame_settings)
    save_flame(img, 'animated_flame_%d' % i, renderer=None, **flame_settings)
    for trans in odd_transform:
        for var in trans.variations:
            var.step()
    del cond

  7%|▋         | 224/3189 [00:35<07:38,  6.47it/s]